In [1]:
import dask

In [2]:
import dask.dataframe as dd

In [12]:
import pandas as pd
import sys

In [13]:
sys.path.append('/Users/pradap/Documents/Research/Python-Package/scaling/dmagellan/')

In [70]:
dda = dd.read_csv('movies.csv', blocksize=10000000)

In [71]:
p0 = dda.get_partition(0)

In [73]:
dda

,id,title,year,length,budget,rating,votes,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,mpaa,Action,Animation,Comedy,Drama,Documentary,Romance,Short
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,
None,int64,object,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,int64,int64,int64,int64,int64,int64,int64
None,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [53]:
type(dda)

dask.dataframe.core.DataFrame

In [54]:
for x in p0.itertuples():
    print(x[1:])
    break
    pass

(0, '#', 2012, 15.0, 10000.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0, 0, 1, 0, 0, 0, 1)


In [55]:
%time xx = dda.map_partitions(preprocess_table)

CPU times: user 12.2 ms, sys: 3.24 ms, total: 15.4 ms
Wall time: 13.6 ms


In [56]:
help(dda.map_partitions)

Help on method map_partitions in module dask.dataframe.core:

map_partitions(self, func, *args, **kwargs) method of dask.dataframe.core.DataFrame instance
    Apply Python function on each DataFrame partition.
    
    Parameters
    ----------
    func : function
        Function applied to each partition.
    args, kwargs :
        Arguments and keywords to pass to the function. The partition will
        be the first argument, and these will be passed *after*.
    meta : pd.DataFrame, pd.Series, dict, iterable, tuple, optional
        An empty ``pd.DataFrame`` or ``pd.Series`` that matches the dtypes
        and column names of the output. This metadata is necessary for
        many algorithms in dask dataframe to work.  For ease of use, some
        alternative inputs are also available. Instead of a ``DataFrame``,
        a ``dict`` of ``{name: dtype}`` or iterable of ``(name, dtype)``
        can be provided. Instead of a series, a tuple of ``(name, dtype)``
        can be used. 

In [57]:
%time kk = xx.compute()

CPU times: user 7.1 s, sys: 1.92 s, total: 9.02 s
Wall time: 5.95 s


In [58]:
from dmagellan.core.stringcontainer import StringContainer
import string
def get_str_cols(dataframe):
    return dataframe.columns[dataframe.dtypes == 'object']
def preprocess_table(dataframe):
    str_cols = list(get_str_cols(dataframe))
    proj_df = dataframe[str_cols]
    concat_strings = []
    str_container = StringContainer()
    str2bytes = lambda x: x if isinstance(x, bytes) else x.encode('utf-8')
    for row in proj_df.itertuples():
        idx = row[0]
        column_values = row[1:]
        strs = [column_value.strip() for column_value in column_values if not pd.isnull(column_value)]
        joined_row = ' '.join(strs)
        joined_row = joined_row.translate(None, string.punctuation)
        concat_strings.append(joined_row.lower())
        str_container.push_back(str2bytes(joined_row.lower()))
    return str_container

In [59]:
strcols = list(get_str_cols(A))

In [60]:
proj_df = dda.get_partition(0)[strcols]

KeyError: "[u'episode' u'song' u'artists'] not in index"